# 🖥️ Spherical grids and velocity conversion

In most applications, Parcels works with `spherical` meshes, where longitude and latitude are given in degrees, while depth is given in meters. But it is also possible to use `flat` meshes, where longitude and latitude are given in meters (note that the dimensions are then still called `longitude` and `latitude` for consistency reasons).

In all cases, velocities are given in m/s. Parcels seamlessly converts between meters and degrees, under the hood. For transparency, this guide explains how this works.


Let's first import the relevant modules, and generate a simple dataset on a 2D spherical mesh, with `U`, `V` and `temperature` data arrays, with the velocities 1 m s<sup>-1</sup> and the temperature 20°C.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

In [ ]:
from parcels._datasets.structured.generated import simple_UV_dataset

nlat = 10
nlon = 18
ds = simple_UV_dataset(dims=(1, 1, nlat, nlon), mesh="spherical")
ds["temperature"] = ds["U"] + 20  # add temperature field of 20 deg
ds["U"].data[:] = 1.0  # set U to 1 m/s
ds["V"].data[:] = 1.0  # set V to 1 m/s
display(ds)

fieldset = parcels.FieldSet.from_sgrid_conventions(ds, mesh="spherical")

Plotting the `U` field indeed shows a uniform 1 m s<sup>-1</sup> eastward flow.


In [ ]:
plt.pcolormesh(
    fieldset.U.grid.lon,
    fieldset.U.grid.lat,
    fieldset.U.data[0, 0, :, :],
    vmin=0,
    vmax=1,
    shading="gouraud",
)
plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.colorbar()
plt.show()

However, printing the velocites directly shows something perhaps surprising. Here, we use the square-bracket field-interpolation notation to print the field value at (5W, 40N, 0m depth) at time 0. 

```{note}
Sampling a velocity in Parcels is done by calling the `fieldset.UV` VectorField; see also the section "Sampling U and V separately" below.
```


In [ ]:
time = np.array([0])
z = np.array([0])
lat = np.array([40])
lon = np.array([-5])
u, v = fieldset.UV[time, z, lat, lon]
temp = fieldset.temperature[time, z, lat, lon]

print(f"(u, v) = ({u}, {v})")
print(f"temperature = {temp}")

While the temperature field indeed is 20°C, as we defined, these printed velocities are much smaller.

This is because Parcels converts under the hood from m s<sup>-1</sup> to degrees s<sup>-1</sup>.

Indeed, if we multiply the value of the U field with 1852 \* 60 \* cos(lat) (the number of meters in 1 degree of longitude) and the value of the V field with 1852 \* 60 (the number of meters in 1 degree of latitude), we get the expected 1 m s<sup>-1</sup> for `u` and `v`.


In [ ]:
u, v = fieldset.UV[time, z, lat, lon]

u = u * 1852 * 60 * np.cos(np.deg2rad(lat))
v = v * 1852 * 60
print(f"(u, v) = ({u}, {v})")

assert np.isclose(u, 1.0)
assert np.isclose(v, 1.0)

You can also interpolate the Field to these values directly by using the `eval()` method and setting `applyConversion=False`, as below


In [ ]:
print(
    fieldset.UV.eval(
        time,
        z,
        lat,
        lon,
        apply_conversion=False,
    )
)

## Don't sample U and V separately

Sampling `U` and `V` separately will _not_ convert to degrees s<sup>-1</sup>, so these velocities cannot be used directly for advection on spherical coordinates. This is one of the main reasons to always use the `UV` VectorField for velocity sampling in Parcels.


In [ ]:
for fld in [fieldset.U, fieldset.V]:
    print(f"{fld.name}: {fld.eval(time, z, lat, lon)}")

## Unit conversion for other fields such as diffusivity

For other fields such as diffusivity, Parcels does not apply similar unit conversions when using a `spherical` mesh. For example, if we define a diffusivity field with value 10 m<sup>2</sup> s<sup>-1</sup>, Parcels will not convert this to degrees<sup>2</sup> s<sup>-1</sup> under the hood. 

If you want to work with diffusivity in degrees<sup>2</sup> s<sup>-1</sup> (for example to move particles using a random walk), you will have to convert this yourself in your kernel. 

Note that for the built-in `DiffusionUniformKh`, `AdvectionDiffusionM1` and `AdvectionDiffusionEM`, the conversion is done automatically.